In [16]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [17]:
def create_network(network_input, n_vocab):
    from keras.layers import Lambda
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    ''' Your Code Here '''
    #LSTM layer
    model.add(LSTM(
        512,
        recurrent_dropout=0.3
    ))
    #batch normalization layer
    #model.add(BatchNormalization())
    #dropout layer
    model.add(Dropout(0.3))
    #fully connected layer
    model.add(Dense(256))
    #relu
    model.add(Activation('relu'))
    #batch normalization layer
    model.add(BatchNorm())
    #dropout layer
    model.add(Dropout(0.3))
    #fully connected layer
    model.add(Dense(n_vocab)) #fix this one
    #temperature
    model.add(Lambda(lambda x: x/ .6))
    #softmax
    model.add(Activation('softmax'))
    
    model.compile(loss="categorical_crossentropy", optimizer='adam')
    #model.fit(network_input, epochs=50, verbose=0)
    
    return model


#create_network()

In [18]:
def get_notes():

    notes = []
    for file in glob.glob("midi/jazz2/*.mid"):
        try:
            midi = converter.parse(file)
            print("Parsing %s" % file)
            notes_to_parse = None
        #try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except Exception as inst:# file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            print(inst)
            pass
        
        #print(notes_to_parse)
        for element in notes_to_parse:
            #print(element)
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    pickle.dump(notes, open('notes.p', 'wb'))
    
    #print("notes: ", notes, "\n")
    return notes


def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 16 

    pitchnames = sorted(set(item for item in notes))
    #print("pitchnames: ", pitchnames, "\n")

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    #print("note_to_int: ", note_to_int, "\n")

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        #print("sequence_in: ", sequence_in)
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        #print("network_input: ", network_input)
        network_output.append(note_to_int[sequence_out])
        
    print("network input", network_input)    
    print("network output", network_output)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [15]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    #print("network_input", network_input)
    #print("n_vocab", n_vocab)
    model = create_network(network_input, n_vocab)

 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    ) 
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(network_input, network_output, epochs=5, callbacks=callbacks_list)

train_network()
print("Finished Training")

Parsing midi/jazz2/OrnetteColeman_Chronology_FINAL.mid
Parsing midi/jazz2/StanGetz_BodyAndSoul_FINAL.mid
Parsing midi/jazz2/ColemanHawkins_Stompin'AtTheSavoy_FINAL.mid
Parsing midi/jazz2/KennyWheeler_DoubleVision_FINAL.mid
Parsing midi/jazz2/LeeKonitz_BopGoesToLeesel_FINAL.mid
Parsing midi/jazz2/FreddieHubbard_SpeakNoEvil_FINAL.mid
Parsing midi/jazz2/MilesDavis_Walkin'_FINAL.mid
Parsing midi/jazz2/ArtPepper_Stardust-2_FINAL.mid
Parsing midi/jazz2/LeeKonitz_Crosscurrent_FINAL.mid
Parsing midi/jazz2/VonFreeman_PassItOn_FINAL.mid
Parsing midi/jazz2/MilesDavis_SoWhat_FINAL.mid
Parsing midi/jazz2/MiltJackson_AllTheThingsYouAre_FINAL.mid
Parsing midi/jazz2/PaulDesmond_AloneTogether_FINAL.mid
Parsing midi/jazz2/PhilWoods_CrazyRhythm_FINAL.mid
Parsing midi/jazz2/BennyCarter_SweetLorraine_FINAL.mid
Parsing midi/jazz2/DavidLiebman_BeginTheBeguine_FINAL.mid
Parsing midi/jazz2/DavidLiebman_Milestones_FINAL.mid
Parsing midi/jazz2/GerryMulligan_LineForLyons_FINAL.mid
Parsing midi/jazz2/BobBerg_Natur

KeyboardInterrupt: 

In [19]:
def prepare_sequences_prediction(notes, pitchnames, n_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 4
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    import random
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        
        indices = list(np.nonzero(prediction[0])[0])
        probs = prediction[0]
        index = random.choices(indices, weights=probs, k=1)
        index = index[0]

        #index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [20]:
def generate():
    notes = pickle.load(open('notes.p', 'rb'))
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_prediction(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    
    ### Add a line to load the weights here
    model.load_weights("weights2-improvement-08-6.2601-bigger.hdf5")
    
    
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [21]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    
generate()
print("done")

ValueError: You are trying to load a weight file containing 7 layers into a model with 5 layers.